In [132]:
print ('bla')

bla


In [133]:
#import sys
#!{sys.executable} -m pip install seaborn

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
from os import path
import sys

In [134]:
K=5 # nombre de bras
recompense_centre=5 # récompense moyenne
dispersion_recompense=4 # écart par rapport à la moyenne
dispersion_resultat_par_bras=2.0 # facteur pour l`écart type appliqué à chaque bras lors des tirages

In [135]:
np.random.seed(seed=42)
recompense_moyenne_bras = np.random.random(K)*dispersion_recompense+5
ecart_type_bras = (np.random.random(K)+0.5)*dispersion_resultat_par_bras

In [24]:
recompense_moyenne_bras

array([6.49816048, 8.80285723, 7.92797577, 7.39463394, 5.62407456])

In [136]:
reference = np.flip(np.argsort(recompense_moyenne_bras))
print(f"Ordre des bras décroissant {reference}")

Ordre des bras décroissant [1 2 3 0 4]


In [26]:
ecart_type_bras

array([1.31198904, 1.11616722, 2.73235229, 2.20223002, 2.41614516])

In [27]:
np.random.seed(2023)
tirage=10000
sample = np.zeros((tirage,K))
for i in range(K):
    for j in range(tirage):
        sample[j,i]=np.random.normal(recompense_moyenne_bras[i],ecart_type_bras[i])


In [28]:
plt.figure()
sbn.boxplot(sample)
plt.xlabel("Bras")
plt.ylabel("Récompense")
plt.show()

In [29]:
def tirage(bras):
    # tirage pour un bras
    # entrée : numéro du bras
    # sortie : récompense pour un tirage
    return np.random.normal(recompense_moyenne_bras[bras],ecart_type_bras[bras])

In [30]:
# Ecrire une fonction qui renvoie les numéros de bras alternativement
# exemple 0 -> 1 -> 2 -> 3 -> 4 -> 0 ......
# c'est ici une politique, brute mais une politique néanmoins :)

def RoundRobinPolicy(action_passee,K):
    # Round robin policy : tire un bras l'un après l'autre
    # input : action= derniere action jouée, K nombre de bras
    # output : nouvelle action proposée

    # VOTRE CODE - 1 ligne
    action_futur = (action_passee +1) % K
    
    ## print('action_passe: ', action_passee)
    ## print('K: ', K)
    ## print ('modulo: ', (action_passee +1) % K)
    ## print ('action_futur: ', action_futur)
    
    return action_futur

In [31]:
assert RoundRobinPolicy(1,2) == 0

In [32]:
np.random.seed(2023)
nombre_tirage=100

# Nous allons stocker les résultats pour chaque bras
Q = np.zeros((K)) # permet de stocket le cumul des récompenses
N = np.zeros((K)) # permet de stocker le nombre de fois qu'un bras est tiré

# conserver cet appel pour la suite pour comparer les résultats
# nombre de bras , valeur initial de la moyenne des récompenses, nom de la politique
result = []
action = K-1 # a changer en fonction de votre implémentation. Ma politique renvoie l'action suivante, donc RoundRobinPolicy(K-1,K) va renvoyer l'action 0

for i in range(nombre_tirage):
    action = RoundRobinPolicy(i, K)
    reward = tirage(action)
    result.append( reward )
    Q[ action ] += reward 
    N[ action ] += 1 

In [33]:
result_round_robin = np.cumsum(result)

In [34]:
plt.figure()
plt.plot(result_round_robin)
plt.show()

# Greedy

In [35]:
def greedyPolicy(num_action, mean_reward_per_arm):
    # purpose : choisi le bras qui rapporte le plus
    # input : nombre de bras, moyenne de résultat connu par bras.
    # output : bras choisi

    ## votre code
    #action = mean_reward_per_arm.index(max(mean_reward_per_arm))
    action = np.argmax(mean_reward_per_arm)
    return action

In [34]:
greedyPolicy(5, np.array([0,1,1,0,2]))

4

In [37]:
np.random.seed(2023)
num_action = nombre_tirage
nombre_tirage=100

#Q=  [100] * num_action
#N= [0]* num_action

Q = np.full((K), 10)
N = np.zeros((K))

result = []
for i in range(nombre_tirage):
    action= greedyPolicy(K, Q/(N+1))
    reward= tirage(action)
    result.append(reward)
    Q[action] += reward
    N[action] +=1
    print(f"bras={action}, recompense(int{reward}), moyenne={np.round(Q/N+1)}")
   


/tmp/xpython_6748/3731277505.py:18: RuntimeWarning: divide by zero encountered in true_divide
  print(f"bras={action}, recompense(int{reward}), moyenne={np.round(Q/N+1)}")


bras=0, recompense(int7.431868347630945), moyenne=[18. inf inf inf inf]
bras=1, recompense(int8.440677752859905), moyenne=[18. 19. inf inf inf]
bras=2, recompense(int5.190512231866764), moyenne=[18. 19. 16. inf inf]
bras=3, recompense(int7.914912528474073), moyenne=[18. 19. 16. 18. inf]
bras=4, recompense(int5.377241556129524), moyenne=[18. 19. 16. 18. 16.]
bras=1, recompense(int7.528983803750662), moyenne=[18. 14. 16. 18. 16.]
bras=0, recompense(int9.980713711235706), moyenne=[14. 14. 16. 18. 16.]
bras=0, recompense(int8.388218701228725), moyenne=[12. 14. 16. 18. 16.]
bras=0, recompense(int6.6279191702170515), moyenne=[11. 14. 16. 18. 16.]
bras=3, recompense(int0.5203021099170826), moyenne=[11. 14. 16. 10. 16.]
bras=1, recompense(int7.601279048756534), moyenne=[11. 12. 16. 10. 16.]
bras=0, recompense(int6.070869316140063), moyenne=[10. 12. 16. 10. 16.]
bras=1, recompense(int7.647068436134308), moyenne=[10. 11. 16. 10. 16.]
bras=1, recompense(int8.327012391522507), moyenne=[10. 10. 16.

In [38]:
# stocker le résultat dans un tableau avec cumul
result_greedy_10 = np.cumsum(result)

In [40]:
# comparer les courbes round robin et greedy optimiste
plt.figure()
plt.plot(result_round_robin, label='Greedy optimistic')
plt.plot(result_greedy_10, label='Greedy optimistic')
plt.legend()
plt.show()

# E-Greedy (epslon gredy)

In [41]:
def egreedyPolicy(num_action,epsilon,mean_reward_per_arm):

    # votre code
    # vous allez tirer un nombre aléatoire (uniforme)si celui est < epsilon, vous allez choisir une action au hasard, sinon 
    # vous choisissez la meilleur action connu.
    
    # Generate a random number to decide between exploration or exploitation
    if np.random.uniform(0, 1) < epsilon:
        # Exploration: choose a random action
        #print (' random < epsilon')
        action = np.random.choice(num_action)
    else:
        # Exploitation: choose the greedy action (the action with the highest mean reward)
        #print (' random >= epsilon')
        action = np.argmax(mean_reward_per_arm)
    
    #print(f"action {action}")
    return(action)

In [39]:
egreedyPolicy(3,0, [0,1,0])

1

In [43]:
assert egreedyPolicy(3,0,[ 0,1,0]) == 1

In [51]:
np.random.seed(2023)
nombre_tirage=100
epsilon=0.5

result=[]

for i in range(nombre_tirage):
    action= egreedyPolicy(K, epsilon, Q)
    reward =  tirage(action)
    result.append(reward)
    
    # Update N (number of pulls for this action)
    N[action] += 1
    
    # Update Q (mean reward estimate for the chosen action)
    Q[action] = Q[action] + (1 / N[action]) * (reward - Q[action])
    
    #print(f"bras={action}, recompense(int{reward}), moyenne={np.round(Q/N+1)}")



In [52]:
# stocker le résultat dans un tableau avec cumul
result_epsilongreedy = np.cumsum(result)

In [53]:
# comparer les courbes round robin et greedy optimiste
plt.figure()
plt.plot(result_round_robin, label='Round Robin')
plt.plot(result_greedy_10, label='Greedy optimistic')
plt.plot(result_epsilongreedy, label='Epsilon Greedy ')
plt.legend()
plt.show()

# Epsilon Greedy dDecay

In [54]:
import numpy as np

np.random.seed(2023)
nombre_tirage = 100  # Total number of iterations
K = 10  # Number of arms
epsilonMax = 0.90  # Starting epsilon (high exploration)
epsilonMin = 0.05  # Minimum epsilon (low exploration)
NbPas = 80        # Number of steps for epsilon decay

result = []
eps = []  # To store epsilon values over time
Q = np.zeros(K)  # Mean reward estimates for each arm (initialize to 0)
N = np.zeros(K)  # Number of times each arm is selected (initialize to 0)

epsilon = epsilonMax  # Initialize epsilon to maximum value

for i in range(nombre_tirage):
    action= egreedyPolicy(K, epsilon, Q)
    
    # Simulate reward for the chosen action (you can modify this logic as needed)
    reward = np.random.randn() + (action + 1)  # Simulated reward
    
    # Update the total reward for the chosen action
    result.append(reward)
    
    # Update Q (mean reward estimate for the chosen action) using incremental formula
    N[action] += 1
    Q[action] = Q[action] + (1 / N[action]) * (reward - Q[action])
    
    # Adjust epsilon with linear decay
    epsilon = max(epsilonMin, epsilonMax - (i / NbPas) * (epsilonMax - epsilonMin))
    eps.append(epsilon)

# At the end of the loop:
# - 'result' contains the rewards collected over time
# - 'eps' contains the epsilon values over time (showing the decay)
# - 'Q' contains the final reward estimates for each arm


In [65]:
print(f'eps {eps}')

eps [0.9, 0.889375, 0.87875, 0.868125, 0.8575, 0.846875, 0.83625, 0.825625, 0.8150000000000001, 0.8043750000000001, 0.7937500000000001, 0.7831250000000001, 0.7725, 0.7618750000000001, 0.75125, 0.7406250000000001, 0.73, 0.7193750000000001, 0.70875, 0.698125, 0.6875, 0.676875, 0.66625, 0.655625, 0.645, 0.634375, 0.62375, 0.613125, 0.6025, 0.591875, 0.58125, 0.570625, 0.56, 0.5493750000000001, 0.5387500000000001, 0.528125, 0.5175000000000001, 0.506875, 0.49625, 0.48562500000000003, 0.47500000000000003, 0.4643750000000001, 0.45375000000000004, 0.44312500000000005, 0.4325, 0.42187500000000006, 0.41125000000000006, 0.400625, 0.39, 0.379375, 0.36875, 0.358125, 0.34750000000000003, 0.33687500000000004, 0.32625000000000004, 0.31562500000000004, 0.30500000000000005, 0.29437500000000005, 0.28375000000000006, 0.27312499999999995, 0.26250000000000007, 0.25187500000000007, 0.24125000000000008, 0.23062500000000008, 0.21999999999999997, 0.2093750000000001, 0.1987500000000001, 0.188125, 0.1775000000000

In [55]:
result_epsilongreedy_decay = np.cumsum(result)

In [58]:
# comparer les courbes round robin et greedy optimiste
plt.figure()
plt.plot(result_round_robin, label='Round Robin')
plt.plot(result_greedy_10, label='Greedy optimistic')
plt.plot(result_epsilongreedy, label='Epsilon Greedy ')
plt.plot(result_epsilongreedy_decay, label='Epsilon Greedy Decay')
plt.legend()
plt.show()

# LinUCB

In [130]:
import numpy as np

def linUCBPolicy(mean_reward_arm, arm_usage, c, num_tirage):
    num_arms = len(mean_reward_arm)
    ucb_values = np.zeros(num_arms)
    
    print (f'num_arms {num_arms}')
    print(f'arm_usage {len(arm_usage)}')
    
    for a in range(num_arms):
        print('----------')
        print(f'iter {a}: ')
        # If the arm has been used, calculate the exploration term
        if arm_usage[a] > 0:
            print(f'arm {a} has already been used')
            exploration_term = np.sqrt((np.log(num_tirage + 1)) / (1 + arm_usage[a]))
        else:
            # Encourage exploration with a large finite value instead of 'inf'
            print(f'arm {a} has not yet been used')
            exploration_term = float('inf')  # A large number to encourage exploration
        
        # Calculate the UCB value
        ucb_values[a] = mean_reward_arm[a] + c * exploration_term
        
    # Select the arm with the highest UCB value (valid index)
    action = np.argmax(ucb_values)
    
    print(f'ucb_values: {ucb_values}')
    
    return action


In [126]:
import numpy as np

# Define the current state of the bandit problem
mean_reward_arm = np.array([1.0, 1.2, 0.8])  # Estimated mean rewards for 3 arms
arm_usage = np.array([5, 10, 2])  # Number of times each arm has been used
c = 1.0  # Exploration parameter
num_tirage = 20  # We are at the 20th pull (num_tirage)

# Call the linUCBPolicy function
chosen_arm = linUCBPolicy(mean_reward_arm, arm_usage, c, num_tirage)

# Output the selected arm
print("Chosen arm:", chosen_arm)


num_arms 3
arm_usage 3
----------
iter 0: 
arm 0 has already been used
----------
iter 1: 
arm 1 has already been used
----------
iter 2: 
arm 2 has already been used
ucb_values: [1.71233448 1.72609388 1.80739308]
Chosen arm: 2


In [87]:
K

10

In [138]:
np.random.seed(2023)
nombre_tirage=10

Q = np.zeros((K))  # Estimated total rewards
N = np.zeros((K))  # Number of times each arm has been used

result=[]
c=0.5

# Initialize with small random values for better differentiation
mean_reward_arm = np.random.rand(K) * 0.1  
recompense_moyenne_bras = mean_reward_arm

num_tirage = nombre_tirage
arm_usage = np.zeros(K)

for i in range(nombre_tirage):
    action=  chosen_arm = linUCBPolicy(mean_reward_arm, arm_usage, c, num_tirage)
    reward = tirage(action)
    print(f'i {i},  action: {action}, reward: {reward}')
    Q[action] += reward
    N[action] += 1
    
    arm_usage[action] += 1  # Update usage count for the chosen arm
    
    # Update the mean reward for the chosen arm
    if N[action] > 0:
        mean_reward_arm[action] = Q[action] / N[action]
        
    result.append(reward)   
    


num_arms 5
arm_usage 5
----------
iter 0: 
arm 0 has not yet been used
----------
iter 1: 
arm 1 has not yet been used
----------
iter 2: 
arm 2 has not yet been used
----------
iter 3: 
arm 3 has not yet been used
----------
iter 4: 
arm 4 has not yet been used
ucb_values: [inf inf inf inf inf]
i 0,  action: 0, reward: -0.5102784754357179
num_arms 5
arm_usage 5
----------
iter 0: 
arm 0 has already been used
----------
iter 1: 
arm 1 has not yet been used
----------
iter 2: 
arm 2 has not yet been used
----------
iter 3: 
arm 3 has not yet been used
----------
iter 4: 
arm 4 has not yet been used
ucb_values: [0.03720386        inf        inf        inf        inf]
i 1,  action: 1, reward: 0.05803959145887493
num_arms 5
arm_usage 5
----------
iter 0: 
arm 0 has already been used
----------
iter 1: 
arm 1 has already been used
----------
iter 2: 
arm 2 has not yet been used
----------
iter 3: 
arm 3 has not yet been used
----------
iter 4: 
arm 4 has not yet been used
ucb_values: [0.037

In [139]:
result_lin_ucb = np.cumsum(result)

In [140]:
# comparer les courbes round robin et greedy optimiste
plt.figure()
plt.plot(result_round_robin, label='Round Robin')
plt.plot(result_greedy_10, label='Greedy optimistic')
plt.plot(result_epsilongreedy, label='Epsilon Greedy ')
plt.plot(result_epsilongreedy_decay, label='Epsilon Greedy Decay')
plt.plot(result_lin_ucb, label='LinUCB')
plt.legend()
plt.show()